In [ ]:
import pandas as pd
import re

df = pd.read_csv('data/output_file.csv')

In [ ]:
df.head(10)

In [ ]:
df['experience'] = df['experience'].astype(str)  # Convert to string to handle NaN values

# Define a function to extract the numerical value from the experience column
def extract_experience(text):
    match = re.search(r'\+?\s*(\d+)', str(text))
    if match:
        return match.group(1)
    else:
        return None

# Apply the function to the 'Expérience' column
df['experience'] = df['experience'].apply(extract_experience)

df.head(5)

In [ ]:


df['salaire'] = df['salaire'].astype(str)


def extract_average_salary(text):
  text = text.replace(' ','')
  matches = re.findall(r'(\d[\d ,.]*)\s*-\s*(\d[\d ,.]*)\s*(?:EUR)?\s*(?:par)?\s*(an|mois)?', str(text), flags=re.IGNORECASE)
  if matches:
    start_salary, end_salary, unit = matches[0]
    start_salary, end_salary = map(float, [start_salary.replace(',','.'), end_salary.replace(',','.')])

    if unit and unit.lower() == 'mois':
        return (start_salary + end_salary) / 2 * 12  # Convert monthly to yearly
    else:
        return (start_salary + end_salary) / 2
  else:
    return None

df['salaire'] = df['salaire'].apply(extract_average_salary)
df['salaire'] = df['salaire'].round(2)
df.head(20)


In [ ]:
average_salaire = df['salaire'].mean()

# Replace NaN values in the 'salaire' column with the average value
df['salaire'].fillna(average_salaire, inplace=True)

In [ ]:
df['salaire'] = df['salaire'].round(0)
df.head()

In [ ]:
df = df[df['salaire'] >= 1000]

In [ ]:
df['location'].replace('Paris 1er - 75', 'Paris - 75', inplace=True)
df['location'].replace('Lyon 1er - 69', 'Lyon - 69', inplace=True)
df[df['location']=='Lyon - 69'].count()

In [ ]:
df[['city', 'department']] = df['location'].str.split(r' - ', expand=True)

# Drop the original 'location' column
df.drop(columns=['location'], inplace=True)


In [ ]:
df[df['department'].isnull()]

In [ ]:
df.shape

In [ ]:
df.dropna(subset=['department'], inplace=True)

In [ ]:
df[df['department'].isnull()]

In [ ]:
import ast

df['skills'] = df['skills'].apply(lambda x: ast.literal_eval(x) if pd.notna(x) else [])


In [ ]:
def remove_duplicates(lst):
    return list(set(lst))

# Apply the function to remove duplicates
df['skills'] = df['skills'].apply(remove_duplicates)

df['skills'] = df['skills'].apply(lambda x: [skill.lower() for skill in x])

In [ ]:
filtered_rows = df[df['skills'].apply(len) > 20]

filtered_rows.shape

In [ ]:
df.to_csv('finalData.csv', index=False)

